# Training stocks model

Steps
* [Create feature vector](#Create-feature-vector)
* [Train locally using mlrun pytorch framework](#Train-locally-using-mlrun-pytorch-framework)

In [1]:
# !pip install -U torch

In [2]:
import mlrun
project = mlrun.get_or_create_project(name='stocks',user_project=True, context="src/")

> 2022-08-09 13:32:04,152 [info] loaded project stocks from MLRun DB


## Create feature vector

In [3]:
# Define the list of features we will be using
features = ['stocks.*',
            'news.sentiment',
            ]

# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'stocks'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          description='stocks information')

# Save the feature vector in the Feature Store
transactions_fv.save()

In [4]:
# Get offline feature vector as dataframe and save the dataset to parquet
import datetime
start_time = datetime.datetime.now()-datetime.timedelta(59)
end_time = datetime.datetime.now()-datetime.timedelta(0)
train_dataset = fstore.get_offline_features(fv_name,start_time=start_time,end_time=end_time, entity_timestamp_column = 'Datetime')
#train_dataset = fstore.get_offline_features(fv_name,with_indexes=True, entity_timestamp_column = 'Datetime')
df = train_dataset.to_dataframe()
df

,Open,High,Low,Close,Volume,ticker2onehot_A,ticker2onehot_AAL,ticker2onehot_AAP,ticker2onehot_AAPL,ticker2onehot_ABBV,ticker2onehot_ABC,ticker2onehot_ABMD,ticker2onehot_ABT,ticker2onehot_ACN,ticker2onehot_ADBE,sentiment
0,377.359985,384.329987,376.320007,382.179993,286897,0,0,0,0,0,0,0,0,0,1,NaN
1,142.229996,143.500000,142.080002,143.229996,20155,0,0,0,0,0,1,0,0,0,0,NaN
2,107.680000,108.070000,107.341003,108.010002,172162,0,0,0,0,0,0,0,1,0,0,NaN
3,14.021000,14.090000,13.930000,14.060000,3230872,0,1,0,0,0,0,0,0,0,0,NaN
4,116.669998,116.925003,116.455002,116.925003,63808,1,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30346,194.289993,194.289993,194.289993,194.289993,0,0,0,1,0,0,0,0,0,0,0,0.0
30347,15.020000,15.020000,15.020000,15.020000,0,0,1,0,0,0,0,0,0,0,0,0.0
30348,132.520004,132.520004,132.520004,132.520004,0,1,0,0,0,0,0,0,0,0,0,0.0
30349,109.000000,109.000000,109.000000,109.000000,0,0,0,0,0,0,0,0,1,0,0,1.0


## Train locally using mlrun pytorch framework 

In [5]:
fn = mlrun.code_to_function('train_stocks_model', kind='job',image='mlrun/ml-models',handler='handler', filename='src/train_stocks.py').apply(mlrun.auto_mount())

In [6]:
import os

params = {'hidden_dim':2,
          'n_layers':1,
          'epochs':3, 
          'vector_name':'stocks',
          'seq_size':5,
          'start_time':59,
          'end_time':0,
          'batch_size':1,
          'model_filepath':os.path.join(os.getcwd(),'src')}

fn.run(local=True,watch=True, params = params)

> 2022-08-09 13:32:04,997 [info] starting run train-stocks-model-handler uid=08df0062bd0245c181ed312813430b19 DB=http://mlrun-api:8080
Epoch 1/3:
Training: 100% |██████████| 30291/30291 [01:01<00:00, 495.06Batch/s, MSELoss=2.09e-6, accuracy=0.999] 
Validating: 100% |██████████| 30291/30291 [00:18<00:00, 1618.21Batch/s, MSELoss=7.53e-5, accuracy=0.991] 

Summary:
+----------+----------------------+
| Metrics  |        Values        |
+----------+----------------------+
| MSELoss  | 0.003544299863278866 |
| accuracy |  0.977447509765625   |
+----------+----------------------+

Epoch 2/3:
Training: 100% |██████████| 30291/30291 [01:01<00:00, 492.27Batch/s, MSELoss=0.00218, accuracy=0.953] 
Validating: 100% |██████████| 30291/30291 [00:18<00:00, 1621.45Batch/s, MSELoss=1.27e-6, accuracy=0.999] 

Summary:
+----------+-----------------------+
| Metrics  |        Values         |
+----------+-----------------------+
| MSELoss  | 0.0010735949035733938 |
| accuracy |  0.9662172198295593   |
+--

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
stocks-dani,...13430b19,0,Aug 09 13:32:05,completed,train-stocks-model-handler,v3io_user=danikind=owner=danihost=jupyter-dani-5fcc55bd99-42fs2,,hidden_dim=2n_layers=1epochs=3vector_name=stocksseq_size=5start_time=59end_time=0batch_size=1model_filepath=/User/test/app-lab/apps/stocks_recreate/src,hidden_dim=2n_layers=1epochs=3vector_name=stocksseq_size=5start_time=59end_time=0batch_size=1model_filepath=/User/test/app-lab/apps/stocks_recreate/srclr=0.0001training_MSELoss=4.5981796574778855e-06training_accuracy=0.9978556632995605validation_MSELoss=0.00040481772157363594validation_accuracy=0.9955461025238037,training_MSELoss.htmltraining_accuracy.htmlvalidation_MSELoss.htmlvalidation_accuracy.htmlMSELoss_summary.htmlaccuracy_summary.htmllr_values.htmlstocks_model_custom_objects_map.jsonstocks_model_custom_objects.zipmodel


> 2022-08-09 13:38:25,067 [info] run executed, status=completed
